In [ ]:
# Spring 환경 구성을 위해 jetbrains의 intellij를 설치하도록 한다.
# 구글에서 jetbrain toolbox를 검색한다.
# https://start.spring.io/로 이동한다.
# 1. Project: Gradle
# 2. Language: Java
# 3. Name: Lecture
# 4. Java: 11
# 5. Dependencies: Spring Boot DevTools, Lombok, 
#    Spring Web, Spring Configuration Processor,
#    Thymeleaf, Spring Data JPA, JDBC API,
#    MySQL Driver, Spring Data JDB를
# 6. 다운로드 받는다.
# 7. unzip lecture.zip으로 압축을 해제한다.
# 8. IntelliJ를 켜서 lecture를 import한다.
# 9. Gradle이 JDK를 찾지 못한다면
#    sudo apt-get update
# 10. 사용할 수 있는 OpenJDK를 찾기 위해 아래 명령을 입력한다.
#     sudo apt-cache search openjdk
# 11. sudo apt-get install openjdk-11-jdk

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [2]:
import numpy as np

In [3]:
from keras.callbacks import EarlyStopping

In [4]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [5]:
X, y = make_classification(
    n_samples = 500, n_features = 16, n_classes = 2
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 0
)

In [6]:
model = Sequential()

# 뉴럴넷을 구성
model.add(Dense(16, activation = 'relu', input_dim = 16))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(4, activation = 'tanh'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(
    optimizer = 'sgd',
    loss = 'binary_crossentropy'
)

# EarlyStopping을 사용한다면
# Fit을 하기 이전에 어디서 중단할지를 정해줘야 한다.
es_callback = EarlyStopping(
    # val_loss: 테스트 로스를 감독함
    monitor = 'val_loss',
    # mode = 'min'은 최소값을 감지함
    mode = 'min'
)
# 콜백이란 ? 이벤트
# 몰컴 성공을 위해 필요한 것
# 1. 부모님이 주무시고 계셔야한다.
# 2. 큰 소리가 나면 안됨
# 3. 부모님이 방문을 열고 나오실때
#    모니터 끄고 자는척 시전

trainRes = model.fit(
    X_train, y_train, 
    validation_data = (X_test, y_test),
    epochs = 400,
    batch_size = 5,
    # 조건이 만족되면 콜백에 의해 fit이 중단되도록 구성한 부분
    callbacks = [es_callback],
    verbose = 0
)

print("Test Loss =", model.evaluate(X_test, y_test))

5/5 [==============================] - 0s 2ms/step - loss: 0.4569
Test Loss = 0.45691341161727905


In [7]:
print(trainRes.history['loss'])

[0.6727380156517029, 0.6047642230987549, 0.5660682320594788, 0.5352090001106262, 0.5077705979347229, 0.4831637442111969, 0.4615618884563446, 0.4419502913951874, 0.42603346705436707, 0.41032761335372925, 0.39720165729522705, 0.3836665451526642, 0.3723462224006653, 0.3623003363609314, 0.3534691631793976, 0.3453436493873596, 0.3371499180793762]


In [8]:
print(trainRes.history['val_loss'])

[0.6171998977661133, 0.5765297412872314, 0.552428662776947, 0.5347570180892944, 0.5188183188438416, 0.5069524049758911, 0.49553075432777405, 0.4874177575111389, 0.48094436526298523, 0.4740760326385498, 0.4678286612033844, 0.465578556060791, 0.45858708024024963, 0.4555664658546448, 0.45355403423309326, 0.4529215097427368, 0.4569133520126343]


In [11]:
import json

In [12]:
pyRes = dict(trainRes.history)
json.dumps(pyRes)

'{"loss": [0.6727380156517029, 0.6047642230987549, 0.5660682320594788, 0.5352090001106262, 0.5077705979347229, 0.4831637442111969, 0.4615618884563446, 0.4419502913951874, 0.42603346705436707, 0.41032761335372925, 0.39720165729522705, 0.3836665451526642, 0.3723462224006653, 0.3623003363609314, 0.3534691631793976, 0.3453436493873596, 0.3371499180793762], "val_loss": [0.6171998977661133, 0.5765297412872314, 0.552428662776947, 0.5347570180892944, 0.5188183188438416, 0.5069524049758911, 0.49553075432777405, 0.4874177575111389, 0.48094436526298523, 0.4740760326385498, 0.4678286612033844, 0.465578556060791, 0.45858708024024963, 0.4555664658546448, 0.45355403423309326, 0.4529215097427368, 0.4569133520126343]}'

In [14]:
import requests

from flask import Flask, jsonify
# pip install flask_restful
from flask_restful import Resource, Api, reqparse

In [ ]:
app = Flask(__name__)
api = Api(app)

# localhost:8080/dataServer로 요청을 하면
# 아래 함수가 동작을 한다는 의미!
@app.route('/dataServer')
def PythonServerResponse():
    print("It's operate")
    summary = {'name': 'Hello', 'major': 'C++'}
    pyRes = dict(trainRes.history)
    return jsonify(pyRes)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jul/2020 12:11:12] "GET /dataServer HTTP/1.1" 200 -


It's operate


127.0.0.1 - - [14/Jul/2020 12:13:44] "GET /dataServer HTTP/1.1" 200 -


It's operate


...

It's operate


127.0.0.1 - - [14/Jul/2020 15:43:31] "GET /dataServer HTTP/1.1" 200 -


It's operate


127.0.0.1 - - [14/Jul/2020 15:43:54] "GET /dataServer HTTP/1.1" 200 -


It's operate


In [ ]:
# sudo apt-get install net-tools
# 설치후
# ifconfig를 터미널에 입력한다.